## Running the 2x2 Simulation chain on Polaris using Singularity container

In [ ]:
#Obtain a debug node to do our work
qsub -I -l select=1 -l filesystems=home:eagle -l walltime=1:00:00 -q debug -A ALCF_for_DUNE

In [ ]:
# Module load command for software loading
module use /soft/modulefiles
module load spack-pe-base/0.6.1
module load apptainer

#load python
# module use /soft/modulefiles ; module load conda; conda activate base

In [ ]:
mkdir workdir
cd workdir

git clone https://github.com/cvmfs/cvmfsexec
cd cvmfsexec/
./makedist osg
cd ..

#change this to you rcvmfsexec path
export CVMFS_PATH=<path to cvmfsexec just cloned>
#export CMVMFS_DIR=/home/fathimamaha/workdir/cvmfsexec

export SINGULARITY_BIND=/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE,$CVMFS_PATH/dist/cvmfs:/cvmfs

#obtain a singularity
./cvmfsexec/cvmfsexec config-osg.opensciencegrid.org dune.opensciencegrid.org fermilab.opensciencegrid.org larsoft.opensciencegrid.org dune.osgstorage.org -- singularity exec docker://opensciencegrid/osgvo-el7 bash

#inside the container

#optional
unset LD_PRELOAD

https://wiki.dunescience.org/wiki/G4beamlineVersion3.06

In [ ]:
#Protodune SP simulation

#inside the container

source /cvmfs/dune.opensciencegrid.org/products/dune/setup_dune.sh
setup dunesw v09_65_01d00 -q e20:prof

cat > tmpgen_sp.fcl << EOF
#include "mcc12_gen_protoDune_beam_cosmics_p1GeV.fcl"
physics.producers.generator.FileName:"/grand/ALCF_for_DUNE/far_detector_data/H4_v34b_1GeV_-27.7_10M_1.root"
EOF

#make sure tmpgen.fcl has the contents
cat tmpgen_sp.fcl

lar -n 1 -c tmpgen.fcl -o gen.root
lar -n 1 -c protoDUNE_refactored_g4_stage1.fcl gen.root -o g4_stage1.root
lar -n 1 -c protoDUNE_refactored_detsim_stage1.fcl g4_stage2.root -o
detsim_stage1.root
lar -n 1 -c protoDUNE_refactored_detsim_stage2.fcl detsim_stage1.root -o
detsim_stage2.root
lar -n 1 -c protoDUNE_refactored_reco_35ms_sce_datadriven_stage1.fcl
detsim_stage2.root -o reco_stage1.root
lar -c eventdump.fcl reco_stage1.root >& eventdump_output.txt
config_dumper -P reco_stage1.root >& config_output.txt
product_sizes_dumper -f 0 reco_stage1.root >& productsizes.txt



In [ ]:
#Protodune HD simulation

#inside the container

source /cvmfs/dune.opensciencegrid.org/products/dune/setup_dune.sh
setup dunesw v09_65_01d00 -q e20:prof

cat > tmpgen_hd.fcl << EOF
#include "prod_beam_cosmics_1GeV_protodunehd.fcl"
physics.producers.generator.FileName:"/grand/ALCF_for_DUNE/far_detector_data/H4_v34b_1GeV_-27.7_10M_1.root"
EOF

#make sure tmpgen.fcl has the contents
cat tmpgen_hd.fcl

lar -c tmpgen.fcl -o hd_gen.root
lar -c standard_g4_protodunehd_stage1.fcl hd_gen.root -o hd_g4_stage1.root
lar -c standard_g4_protodunehd_stage2.fcl hd_g4_stage1.root -o
hd_g4_stage2.root
lar -c standard_detsim_protodunehd.fcl hd_g4_stage2.root -o
hd_detsim_stage1.root
lar -c standard_reco_protodunehd.fcl hd_detsim_stage1.root -o
hd_reco_stage1.root
lar -c eventdump.fcl hd_reco_stage1.root >& hd_eventdump_output.txt
config_dumper -P hd_reco_stage1.root >& hd_config_output.txt
product_sizes_dumper -f 0 hd_reco_stage1.root >& hd_productsizes.txt

